In [166]:
import numpy as np
from bs4 import BeautifulSoup
import urllib3
import multiprocessing
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
import json

In [25]:
client = MongoClient()
db = client['dota']

In [27]:
pro_matches_id_coll = db.pro_matches_id

In [151]:
def get_working_proxy():
    user_agent = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64;en; rv:5.0) \
                  Gecko/20110619 Firefox/5.0'}
    proxy_list_site = 'https://free-proxy-list.net/'
    pm = urllib3.PoolManager(1, 
                             headers=user_agent, 
                             cert_reqs='CERT_REQUIRED')
    prx_req = pm.request('GET', proxy_list_site)
    prx_soup = BeautifulSoup(prx_req.data, 'html.parser')
    proxies_list = []
    for tr in prx_soup.find_all('tr'):
        tmp = []
        for td in tr.find_all('td')[:2]:
            tmp.append(td.text)
        if len(tmp) < 2:
            continue
        proxies_list.append([str(tmp[0]) + ':' + tmp[1]])
    
    proxies_list = proxies_list[:20]
    np.random.shuffle(proxies_list)
    for i, prx in enumerate(proxies_list):
        try:
            print('Getting working proxy...')
            cur_prx_address = 'https://' + str(prx[0]) + '/'
            prx_http = urllib3.ProxyManager(cur_prx_address,
                                            maxsize=1, 
                                            headers=user_agent, 
                                            cert_reqs='CERT_REQUIRED')
            r = prx_http.request('GET', 'https://stackoverflow.com/', timeout=0.7)
            if r.status == 200:
                print('Proxy found.')
                return cur_prx_address
            time.sleep(0.1)
        except Exception as err:
            # print(err)
            continue

In [152]:
def api_pro_matches_id(prx_address, start_id=9994966018):
    url = 'https://api.opendota.com/api/proMatches?less_than_match_id='
    user_agent = {'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64;en; rv:5.0) \
                  Gecko/20110619 Firefox/5.0'}
    cur_url = url + str(start_id)
    prx_m = urllib3.ProxyManager(prx_address, headers=user_agent, cert_reqs='CERT_REQUIRED')
    r = prx_m.request('GET', cur_url, timeout=1)
    matches = json.loads(r.data)
    return matches

In [171]:
last_id = 3471469927
cur_prx = get_working_proxy()
for i in range(1000):
    total_matches = pro_matches_id_coll.count()
    print(i, total_matches, last_id) if i % 10 == 0 else None
    try:
        cur_matches = api_pro_matches_id(cur_prx, start_id=last_id)
        ids = [x['match_id'] for x in cur_matches]
        if hasattr(cur_matches, 'error') and cur_matches['error'] == 'rate limit exceeded':
            cur_prx = get_working_proxy()
            continue
        last_id = np.min(ids)
        pro_matches_id_coll.insert(cur_matches)
        total_matches = pro_matches_id_coll.count()
    except DuplicateKeyError as de:
        continue
    except Exception as err:
        # raise
        print(err)
        continue

Getting working proxy...
Getting working proxy...
Getting working proxy...
Getting working proxy...
Getting working proxy...
Getting working proxy...
Getting working proxy...
Getting working proxy...
Getting working proxy...
Getting working proxy...
Getting working proxy...
Getting working proxy...
Proxy found.
0 17600 3471469927


/home/nemo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  del sys.path[0]


10 17600 3406443796
20 17600 3282263076
30 17600 3202257413
40 17600 3155196480
50 17600 3083107298
60 17600 3030922195
70 17600 2920512320
80 17600 2836078773
90 17600 2762373121
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
100 17600 2753112411
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
string indices must be integers
110 17600 2723373532
120 17600 2665388881
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
HTTPSConnectionPool(host='api.opendota.com', port=443): Max retries exceeded with url: /api/proMatches?less_than_match_id=2657008002 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
HTTPSConnectionPool(host='api.opendota.com', port=